[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/69_turkey_earthquake.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/69_turkey_earthquake.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**Visualizing Maxar Open Data for the 2023 Turkey-Syria Earthquake**

The [Maxar Open Data Program](https://www.maxar.com/open-data) provides pre- and post-event high-resolution satellite imagery in support of emergency planning, risk assessment, monitoring of staging areas and emergency response, damage assessment, and recovery. Check out the links below for more information.
- [Maxar Open Data Program](https://www.maxar.com/open-data)
- [Maxar Open Data on AWS](https://registry.opendata.aws/maxar-open-data/)
- [Maxar Open Data on STAC Index](https://stacindex.org/catalogs/maxar-open-data-catalog-ard-format#/)
- [Maxar Open Data on STAC Browser](https://radiantearth.github.io/stac-browser/#/external/maxar-opendata.s3.amazonaws.com/events/catalog.json?.language=en)
- [Maxar Open Data in CSV, GeoJSON, and MosaicJSON formats](https://github.com/giswqs/maxar-open-data)

This notebook shows how to visualize and download the Maxar Open Data for the 2023 Turkey Earthquake using leafmap.

First, install libraries and import modules.

In [ ]:
# !pip install -U leafmap geopandas

In [ ]:
import leafmap
import geopandas as gpd

Retrieve all collections from the Maxar Open Data STAC catalog. Each collection represents a single event.

In [ ]:
leafmap.maxar_collections()

The collection ID for the 2023 Turkey Earthquake is `Kahramanmaras-turkey-earthquake-23`. We can get the footprints ([geojson](https://github.com/giswqs/maxar-open-data/blob/master/datasets/Kahramanmaras-turkey-earthquake-23.geojson), [tsv](https://github.com/giswqs/maxar-open-data/blob/master/datasets/Kahramanmaras-turkey-earthquake-23.tsv)) of the event from the [Maxar Open Data GitHub repo](https://github.com/giswqs/maxar-open-data):

In [ ]:
collection = "Kahramanmaras-turkey-earthquake-23"
url = leafmap.maxar_collection_url(collection, dtype="geojson")
url

Let's find out how many images are available for the event:

In [ ]:
gdf = gpd.read_file(url)
print(f"Total number of images: {len(gdf)}")
gdf.head()

Visualize the footprints of the images on the map:

In [ ]:
m = leafmap.Map()
m.add_gdf(gdf, layer_name="Footprints")
m

The earthquake started on February 6, 2023. We can use the `start_date` and `end_date` parameters to filter the images by date. Set `end_date` to `2023-02-06` to get all images captured before the earthquake:

In [ ]:
pre_gdf = leafmap.maxar_search(collection, end_date="2023-02-06")
print(f"Total number of pre-event images: {len(pre_gdf)}")
pre_gdf.head()

Set the `start_date` to `2023-02-06` to get all images captured after the earthquake:

In [ ]:
post_gdf = leafmap.maxar_search(collection, start_date="2023-02-06")
print(f"Total number of post-event images: {len(post_gdf)}")
post_gdf.head()

Visualize the pre-event and post-event image footprints on the map. The red footprints represent the pre-event images and the blue footprints represent the post-event images:

In [ ]:
m = leafmap.Map()
pre_style = {"color": "red", "fillColor": "red", "opacity": 1, "fillOpacity": 0.5}
m.add_gdf(pre_gdf, layer_name="Pre-event", style=pre_style, info_mode="on_click")
m.add_gdf(post_gdf, layer_name="Post-event", info_mode="on_click")
m

Use the draw tools to select a region of interest (ROI) and get the bounding box coordinates:

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [36.8715, 37.5497, 36.9814, 37.6019]

Search the Maxar Open Data catalog for images within the ROI before the earthquake:

In [ ]:
pre_event = leafmap.maxar_search(collection, bbox=bbox, end_date="2023-02-06")
pre_event.head()

Search the Maxar Open Data catalog for images within the ROI after the earthquake:

In [ ]:
post_event = leafmap.maxar_search(collection, bbox=bbox, start_date="2023-02-06")
post_event.head()

Get the catalog id of the pre-event tile, which contain several images. Each of the image has a unique `quadkey`:

In [ ]:
pre_tile = pre_event["catalog_id"].values[0]
pre_tile

Get the catalog id of the post-event tile, which contain several images. Each of the image has a unique `quadkey`:

In [ ]:
post_tile = post_event["catalog_id"].values[0]
post_tile

Get the MosaicJSON for the pre-event tile:

In [ ]:
pre_stac = leafmap.maxar_tile_url(collection, pre_tile, dtype="json")
pre_stac

Get the MosaicJSON for the post-event tile:

In [ ]:
post_stac = leafmap.maxar_tile_url(collection, post_tile, dtype="json")
post_stac

Create a split map to compare the pre-event and post-event images. Note the the ipyleaflet split map has some bugs ([source](https://github.com/jupyter-widgets/ipyleaflet/issues/1066)), so we will use the folium plotting backend instead.

In [ ]:
import leafmap.foliumap as leafmap

In [ ]:
m = leafmap.Map()
m.split_map(
    left_layer=pre_stac,
    right_layer=post_stac,
    left_label="Pre-event",
    right_label="Post-event",
)
m.set_center(36.9265, 37.5762, 16)
m

Get download links for the pre-event and post-event images:

In [ ]:
pre_images = pre_event["visual"].tolist()
post_images = post_event["visual"].tolist()

Download the pre-event and post-event images:

In [ ]:
leafmap.maxar_download(pre_images)

In [ ]:
# leafmap.maxar_download(post_images)